## KoNLPy 형태소 분석기 선정

### 대상
1. hannanum
2. kkma
3. okt

### 선정 방법

책 10권에 대해 각각 10개의 키워드 선정
hannanum,kkma,okt 명사추출 기능으로 해당 키워드가 구분되는지 체크 

### 도서목록
1. 리액트를 다루는 기술 = The art of React : 실무에서 알아야 할 기술은 따로 있다!

2. (Do it!) 장고 + 부트스트랩 파이썬 웹 개발의 정석 : 만들면서 배우는 웹 개발 A to Z
3. 머신러닝 디자인 패턴 : 효율적인 머신러닝 파이프라인과 MLOps를 구축하는 30가지 디자인 패턴
4. 핸즈온 머신러닝
5. 파이썬을 이용한 데이터 분석의 정석 : 넘파이, 판다스, 맷플롯립과 실전 예제로 배우는
6. 스파크 완벽 가이드 : 스파크를 활용한 빅데이터 처리와 분석의 모든 것
7. 처음 시작하는 딥러닝
8. 컴퓨터 사이언스 부트캠프 with 파이썬
9. 케라스 창시자에게 배우는 딥러닝
10. 처음 배우는 리액트 네이티브 크로스 플랫폼 앱 개발을 위한 실전 입문서


In [1]:
from konlpy.tag import Hannanum,Okt,Kkma
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec,KeyedVectors
import re

### 성능 비교에 사용 될 도서 10권 추출

In [2]:
bookinfo = pd.read_csv('./data/bookInfo12.csv',encoding='cp949')
testBooksISBN = [9791160508796,9791163032069,9791162244845,9791162242964,9791186710708,9791162241288,9791162243343,9791160504095,9791160505979,9791162243879]
testBooks = bookinfo[bookinfo['col1'].isin(testBooksISBN)]
han = Hannanum()


def mergeListToString(wordList:list) -> str :
    '''
    wordList = item.astype(str).tolist()
    item = pd.Series()
    '''
    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))
    result :str = ' '.join(str_list)
    return result

In [16]:
bookinfo = pd.read_csv('./data/keywordExtraction.csv',encoding='cp949')

bookinfo

,Unnamed: 0,도서명,저자,출판사,ISBN,주제분류번호,등록일자,이미지주소,keyword,TFIDFKeyword
0,0,(그림과 실습으로 배우는) 도커 & 쿠버네티스:개념과 작동 원리가 쏙쏙 이해되는 완...,오가사와라 시게타카 지음 ;심효섭 옮김,위키북스,9791158393038,005.1,2022-06-22,http://image.kyobobook.co.kr/images/book/large...,['도커' '실습' '컨테이너' '쿠버네티스' '설치' '실행' '사용법' '컴포즈...,"['도커', '컨테이너', '실습', '쿠버네티스', '컴포즈', '매니페스트', ..."
1,1,도메인 주도 설계 첫걸음:소프트웨어 아키텍처와 비즈니스 전략의 일치를 위한 핵심 패...,"블라드 코노노프 지음;김민석,오창윤 옮김",위키북스,9791158393359,005.115,2022-06-22,http://image.kyobobook.co.kr/images/book/large...,['도메인' '설계' '비즈니스' '컨텍스트' '모델' '바운디드' '결론' '연습...,"['도메인', '바운디드', '컨텍스트', '결론', '비즈니스', '이벤트스토밍'..."
2,2,"(눈 떠보니) 메타버스 마스터 :메타버스 플랫폼(이프랜드, 제페토, 게더타운) 활용...","최재용,진성민 지음",光文閣,9788970936208,004.783,2022-06-15,http://image.kyobobook.co.kr/images/book/large...,['메타버스' '플랫폼' '활용' '세상' '경제' '영화' '현실' '활동' '게...,"['메타버스', '게더타운', '이프랜드', '가상세계', '플랫폼', '경제', ..."
3,3,혼자 공부하는 얄팍한 코딩 지식,고현민 지음,한빛미디어,9791162245552,005.13,2022-06-15,http://image.kyobobook.co.kr/images/book/large...,['개발자' '용어' '핵심' '포인트' '정리' '확인' '학습' '언어' '프로...,"['포인트', '개발자', '용어', '키워드', '핵심', '혼공', '정리', ..."
4,4,(Do it!) 점프 투 파이썬 :라이브러리 예제 편,박응용 지음,이지스퍼블리싱,9791163033554,005.133,2022-06-15,http://image.kyobobook.co.kr/images/book/large...,['라이브러리' '파이썬' '파일' '데이터' '실행' '서버' '저장' '예제' ...,"['파이썬', '라이브러리', '파일', '서버', '데이터', '저장', '실행'..."
...,...,...,...,...,...,...,...,...,...,...
5232,5232,와이어샤크 트러블슈팅 :네트워크 성능 개선과 문제 해결,로라 채플 지음 ;강지양 옮김,에이콘,9788960778948,004.5,2018-02-22,http://image.kyobobook.co.kr/images/book/large...,['와이어샤크' '실습' '문제' '네트워크' '패킷' '분석' '그래프' '해결'...,"['와이어샤크', '패킷', '실습', '문제', '네트워크', '그래프', '트래..."
5233,5233,이것이 C++이다 :강의 현장을 그대로 옮긴 C++입문서,최호성 지음,한빛미디어,9788968482465,005.133,2018-02-22,http://image.kyobobook.co.kr/images/book/large...,['프로그래밍' '프로그래머' '객체지향' '상속' '연산자' '특징' '공부' '...,"['객체지향', '프로그래밍', '상속', '프로그래머', '널널하', '널널한',..."
5234,5234,(주말에 끝내는)PHP 프로그래밍: 이틀 만에 개발 환경 구축부터 간단한 웹 애플리...,요시타니 아이 지음;김현옥 옮김,프리렉,9788965401278,005.138,2018-02-01,http://image.kyobobook.co.kr/images/book/large...,['프로그래밍' '주말' '표시하기' '페이지' '브라우저' '애플리케이션' '프로...,"['프로그래밍', '표시하기', '페이지', '주말', '게시판', '좌절', '브..."
5235,5235,(창의적인 뇌로 향상시키는) Let*s go! 스크래치 프로그래밍,"허태성,주길홍,신현정,조규철 지음",북스홀릭 Publishing(북스홀릭 퍼블리싱),9791185123912,005.11,2018-01-25,http://image.kyobobook.co.kr/images/book/large...,['게임' '컴퓨팅적' '소프트웨어' '스크래치' '무대' '프로그램' '창의적' ...,"['무대', '컴퓨팅적', '게임', '스크래치', '스프라이트', '구구단', '..."


### TF-IDF

In [ ]:
# list1 = books.iloc[1].tolist()
def TFIDF(wordsList) :
    from operator import itemgetter

    # str_list = [re.sub('\d','',str(a)) for a in wordList]
    # str_list = list(filter(None, str_list))

    # han = Hannanum()
    # word = ' '.join(str_list)
    # konlpword = han.nouns(word)
    # konlpword = ' '.join(konlpword)

    vectorizer = TfidfVectorizer()
    sp_matrix = vectorizer.fit_transform(wordsList)

    # a,b
    a = np.array(vectorizer.get_feature_names_out())
    b = sp_matrix.toarray()[0]
    unique, counts = np.unique(b, return_counts=True)

    ### Top 10 추출
    k = 0
    for num in range(1,100) :
        if k < 10 :
            k += counts[-num]
        else :
            break

    result =[(a,b) for a,b in zip(a,b) if b >= unique[-num]]            
    result = sorted(result,key=itemgetter(1),reverse=True)
    return result

wordsList=[mergeListToString(i[1]) for i in testBooks.iterrows()]
konlpyWordsForTFIDF = list(map(lambda x : ' '.join(han.nouns(x)),wordsList ))
k = TFIDF(konlpyWordsForTFIDF)
TFIDFKeyword = list(map(lambda x : x[0],k))

### TF-IDF 5000개 적용 및 MultiProcessing 사용

In [ ]:
from operator import itemgetter

#도서별 모든 문장 하나로 합치기
wordsList=[mergeListToString(i[1].astype(str).tolist()) for i in bookinfo.iterrows()] 

 # 모든 문장 형태소 분석
konlpyWordsForTFIDF = list(map(lambda x : ' '.join(han.nouns(x)),wordsList ))


# 형태소 분석 결과 비지도학습
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(konlpyWordsForTFIDF) 

# 결과 추출
a = np.array(vectorizer.get_feature_names_out()) # n개 키워드
b = sp_matrix.toarray() ## 책 권수 만큼 array 생성(5237개). / 개별 array shape = shape(n,)

# Keyword Top 10 추출
result = []
for arr in b :
    unique, counts = np.unique(arr, return_counts=True)
    ### Keyword Top 10 추출
    k = 0
    for num in range(1,100) :
        if k < 10 :
            k += counts[-num]
        else :
            break

    try : 
        itemList =[(name,percent) for name,percent in zip(a,arr) if percent >= unique[-num]]            
        itemList = sorted(itemList,key=itemgetter(1),reverse=True)
        result.append(itemList)
    except:
        result.append(None)

# 추출 결과를 column에 넣기 위한 전처리
TFIDFKeyword =[]
for item in result:
    list1=[]
    for x in item:
        list1.append(x[0])
    TFIDFKeyword.append(list1)

In [ ]:
## 개별 단어에 Top 10 확보
# TF-IDF 단점 : 매달 TF-IDF를 돌려야함. 비효율적이라 판단
TFIDFKeyword[0]

['도커', '컨테이너', '실습', '쿠버네티스', '컴포즈', '매니페스트', '커맨드', '파드', '리눅스용', '설치', '실행']

### Word2Vec

### 학습과정

In [ ]:
def mergeListToString(item:pd.Series) :
    wordList = item.astype(str).tolist()
    str_list = [re.sub('\d','',str(a)) for a in wordList]
    str_list = list(filter(None, str_list))
    result = ' '.join(str_list)
    return result

wordsList=[mergeListToString(i[1]) for i in bookinfo.iterrows()]
print('Complete wordsList Load!!')
konlpyWords = list(map(lambda x : han.nouns(x),wordsList ))
embedding_model = Word2Vec(sentences=konlpyWords, window = 2, min_count=50, workers=7, sg=1)

### 학습된 vec 열기

In [21]:
import re

# 문장 들어오면 단어로 잘라주는 메소드
def separateKeyword(words) :

    if type(words) != str :
        raise ValueError('str only possible')

    k = re.findall(r'\s|,|[^,\s]+', words)
    k = [i for i in k if i not in [',',' ']]
    return k

# 영문을 한글로 변환
def transToHan(words:list) -> list:
    EngToKorDict=pd.read_csv('./data/englist.csv',index_col=0)
    result = []
    for word in words :
        enToko = EngToKorDict[EngToKorDict['0'].isin([word])]
        if enToko.empty is not True :
            result.extend(enToko['1'].tolist())
        else :
            result.append(word)
        
    return list(set(result))

# 영문 문자 리스트 추출
def findEng(text: str) -> str:
    return re.findall("[a-zA-Z]+", text)
    
# 한글 문자 리스트 추출
def findHan(text: str) -> str:
    return re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', text)

# 최종 
def searchKeyword(word:list) :
    val = separateKeyword(word)
    keywordItems = transToHan(val)
    engList = list(filter(lambda x : findEng(x),keywordItems))
    hanList = list(filter(lambda x : findHan(x),keywordItems))
    return dict(eng=engList,han=hanList,all=keywordItems)

In [2]:
import ast


def changeStringToList(strList) :
    # strList = strList.replace(' ',', ')
    # result = np.array(ast.literal_eval(strList))
    return ast.literal_eval(strList)

# 중복값 찾기
def findOverlapNum(keywordsOfBook:list,keywordsWord2Vec):
    return np.in1d(keywordsWord2Vec,keywordsOfBook)

# 영문 검색 기능 추가
def searchByKeywords(words:str) :
    #검색 단어 한영 쪼개기
    wordDict : dict = searchKeyword(words)

    #한영 전부 있는 단어
    allList = wordDict['all']
    wordsLen = len(allList)

    #한글만 추출
    hanList = wordDict['han']

    #model load(나중에 밖으로 뺴내야할 듯)
    loaded_model = KeyedVectors.load_word2vec_format("booksTest1")

    #키워드 단어 불러오기 20개 추출
    keywordsWord2Vec = loaded_model.most_similar(positive=hanList,topn=20)
    Word2VecKeyword = list(map(lambda x : x[0],keywordsWord2Vec))

    # 사용자가 검색한 단어와 합치기
    allList.extend(Word2VecKeyword)

    #추출한 키워드를 알고싶다면?
    # print(allList)
    
    # 키워드 일치여부 확인하고 사용자 검색 단어에 가중치 넣기
    val = np.array(list(map(lambda x : findOverlapNum(x,allList) ,result)))
    df = pd.DataFrame(val)
    for i in range(wordsLen) :
        df[i] = df[i]*3
        
    return df.T.sum()

# searchByKeywords 내에 포함되어야 할듯 
dfRaw = pd.read_csv('./data/keywordExtraction2.csv',encoding='cp949',index_col=0)
lists = dfRaw['keywords'].tolist()
result = list(map(changeStringToList,lists))

In [28]:
# df['keywords']의 String을 List로 반환


kkkk=dfRaw.copy()
userkeywords = '데이터베이스, 오라클'
kkkk['sums'] = searchByKeywords(userkeywords)

kkkk[kkkk['sums'] >= 3][['도서명','ISBN','sums']].sort_values(by='sums',ascending=False)

,도서명,ISBN,sums
3189,(그림으로 공부하는)오라클 구조,9791188621996,7
5192,그림으로 공부하는 오라클 구조,9791185890302,7
2881,오라클로 배우는 데이터베이스 개론과 실습,9791156645023,7
1645,(전문가를 위한) 오라클 아키텍처 입문,9791188621101,7
1734,(초보자를 위한) Oracle 12c :DBA 편,9791195942534,7
...,...,...,...
2866,데이터베이스 인터널스 (분산 데이터베이스 시스템 심층 분석),9791161754963,3
2874,AWS 공인 솔류션스 아키텍트 올인원 스터디 가이드,9791161754482,3
666,깔끔한 파이썬 탄탄한 백엔드 :지금까지 없었던 백엔드 개발자를 위한 파이썬,9791186697757,3
2919,(PostgreSQL로 시작하는)SQL 코딩입문=SQL Programming in ...,9788995447468,3


# 이제 이걸 SQl로 가능하게 만들어야겠다.

In [ ]:
def extracKeywords(words:list) :
    #검색 단어 한영 쪼개기
    wordDict = searchKeyword(words)

    #한영 전부 있는 단어
    allList = wordDict['all']
    wordsLen = len(allList)

    #한글만 추출
    hanList = wordDict['han']

    #model load(나중에 밖으로 뺴내야할 듯)
    loaded_model = KeyedVectors.load_word2vec_format("booksTest1")

    #키워드 단어 불러오기 20개 추출
    keywordsWord2Vec = loaded_model.most_similar(positive=hanList,topn=20)
    Word2VecKeyword = list(map(lambda x : x[0],keywordsWord2Vec))

    # 사용자가 검색한 단어와 합치기
    allList.extend(Word2VecKeyword)

    return allList
    
userkeywords = '데이터베이스, sql'

# extracKeywords(userkeywords)

# csv for sql

In [33]:
# csv = pd.read_csv('./data/keywordExtraction2.csv',encoding='cp949',index_col=0)

# lists = csv['keywords'].tolist()
# result = list(map(changeStringToList,lists))

# k = pd.DataFrame(result)

# asd = pd.merge(csv,k,how='left',left_index=True,right_index=True)
# asd = asd.drop(columns='Unnamed: 0')
# asd.to_csv('./data/keywordExtraction2ForSQL.csv')

In [ ]:
csv = pd.read_csv('./data/keywordExtraction2ForSQL.csv',index_col=0)

# csv = csv.drop(columns='39')
# csv.to_csv('./data/keywordExtraction2ForSQL.csv')
csv.head(2)

In [28]:
# k = np.vectorize(len)
# for col in csv.columns :
#     print(f'{col} : ',k(csv[col].astype(str).values).max())
# ---
# for i in range(39):
#     print(f"key{i} = models.CharField(max_length=20),"),

In [31]:
import pandas as pd
import pymysql
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='',db='dash_test')
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [33]:
cursor.execute(' desc backend_dodomoalibinfo')
result = cursor.fetchall()
pd.DataFrame(result)

,Field,Type,Null,Key,Default,Extra
0,id,bigint,NO,PRI,None,auto_increment
1,지역,varchar(5),NO,,None,
2,ISBN,bigint,NO,MUL,None,


### 배운내용 저장(Notion에도 저장했음.)

- WHERE을 사용할 때 Value가 String일 경우 따옴표를 붙여줘야 정상작동한다.

-  isin과 같음 : FIND_IN_SET(column명, 변수) * 변수 = 'a,b,c' 형태여야함 

In [65]:
var = "영등포,동작,양천"
cursor.execute(f'SELECT * FROM backend_dodomoalibinfo WHERE FIND_IN_SET(지역,"{var}") >0')
result = cursor.fetchall()
pd.DataFrame(result).head(3)

,id,지역,ISBN
0,1,영등포,9788970936208
1,2,영등포,9788970505411
2,3,영등포,9791197119927


In [68]:
var = "양천"
cursor.execute(f'SELECT ISBN FROM backend_dodomoalibinfo WHERE FIND_IN_SET(지역,"{var}") >0')
result = cursor.fetchall()
pd.DataFrame(result)

,ISBN
0,9788934961772
1,9788980783106
2,9788934949541
3,9791165219659
4,9788931556698
...,...
1023,9788956747842
1024,9791162240861
1025,9791158740405
1026,9791162241042


### WHERE 순서에 따른 JOIN 속도 비교하기

In [154]:
# JOIN WHERE
# 18 ms ± 538 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

var = "양천"
cursor.execute(f'''SELECT book.*, lib.지역 FROM backend_dodomoabookinfo AS book
                   LEFT JOIN backend_dodomoalibinfo AS lib
                   ON book.ISBN = lib.ISBN
                   where FIND_IN_SET(지역,"{var}") > 0''')
result = cursor.fetchall()
a = pd.DataFrame(result).drop_duplicates(subset=['id','지역'])

In [142]:
# JOIN On
# 54.1 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 10 loops each) 
#### and를 넣으니 FIND_IN_SET이 적용되지 않음.
var = "양천"
cursor.execute(f''' SELECT book.*, lib.지역 FROM backend_dodomoabookinfo AS book
                   LEFT JOIN backend_dodomoalibinfo AS lib
                   ON (book.ISBN = lib.ISBN
                   AND FIND_IN_SET(지역,"{var}") > 0)''')
result = cursor.fetchall()
a = pd.DataFrame(result).drop_duplicates(subset='id')

In [138]:
# WHERE 먼저 그다음 Join
# 15 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

var = "양천"
cursor.execute(f''' SELECT book.*, lib.지역 FROM (SELECT * FROM backend_dodomoalibinfo where FIND_IN_SET(지역,"{var}") > 0) AS lib 
                   LEFT JOIN  backend_dodomoabookinfo AS book
                   ON book.ISBN = lib.ISBN
                   ''')
result = cursor.fetchall()
a = pd.DataFrame(result).drop_duplicates(subset='id')
a.head(2)

,id,ISBN,도서명,저자,출판사,주제분류번호,등록일자,이미지주소,지역
0,8276.0,9.788935e+12,"전길남, 연결의 탄생: 한국 인터넷의 개척자 전길남 이야기",구본권 지음,김영사,004.58,2022-06-07,http://image.kyobobook.co.kr/images/book/large...,양천
1,12574.0,9.788981e+12,(Python™으로 배우는) OpenCV 프로그래밍 =Image Processing...,김동근 지음,KM(가메출판사),004.77,2022-06-14,http://image.kyobobook.co.kr/images/book/large...,양천
